In [1]:
%config InteractiveShell.ast_node_interactivity="last_expr_or_assign"

# Day 4: Squid Game
The input for this problem is located at https://adventofcode.com/2021/day/4/input

In [2]:
import awkward as ak
import numpy as np

In [3]:
def make_index(array):
    for i in range(array.ndim):
        array = ak.from_regular(array, axis=i)
    return array

In [4]:
def as_type(array, dtype):
    return ak.values_astype(ak.regularize_numpyarray(array), dtype)

In [5]:
with open("input.txt") as f:
    draw = ak.from_numpy(np.loadtxt(f, dtype=np.uint64, delimiter=",", max_rows=1))
    board = ak.from_numpy(
        np.loadtxt(f, dtype=np.uint64).reshape(-1, 5, 5), regulararray=True
    )

Evolve state of board through each draw

In [6]:
match = as_type(
    np.cumsum(
        draw[:, np.newaxis, np.newaxis, np.newaxis] == board,
        axis=0,
    ),
    np.bool_,
)

<Array [[[[False, False, ... True, True]]]] type='100 * 100 * 5 * 5 * bool'>

Find unmarked board values

In [7]:
match_board, _ = np.broadcast_arrays(board, match)
unmarked = match_board[~make_index(match)]

<Array [[[[25, 29, 78, 57, 69, ... [], []]]] type='100 * var * var * var * uint64'>

Sum unmarked values for each draw

In [8]:
score = draw * np.sum(np.sum(unmarked, axis=-1), axis=-1)

<Array [[85491, 73332, 55629, ... 0, 0, 0]] type='100 * var * uint64'>

Determine which boards have won

In [9]:
win_by = np.any(np.all(match, axis=-1), axis=-1) | np.any(
    np.all(match, axis=-2), axis=-1
)

<Array [[False, False, False, ... True, True]] type='100 * var * bool'>

Find which boards have won in exactly n moves

In [10]:
win_exact = np.concatenate((win_by[:1], win_by[1:] & ~win_by[:-1]))
score_win = np.ravel(score[win_exact])

<Array [63424, 9720, 9930, ... 7296, 23541] type='100 * uint64'>

In [11]:
score_win[0]

63424

In [12]:
score_win[-1]

23541